In [1]:
#!/usr/bin/env python3
import os, sys
from normalize_scores_utils import *

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from aces.cli.evaluate import comp_aces_score

# Main Part: Table and Correlations

In [71]:
metrics_names_2022

{'BERTScore',
 'BLEU',
 'BLEURT-20',
 'COMET-20',
 'COMET-22',
 'COMET-QE',
 'COMETKiwi',
 'Cross-QE',
 'HWTSC-TLM',
 'HWTSC-Teacher-Sim',
 'KG-BERTScore',
 'MS-COMET-22',
 'MS-COMET-QE-22',
 'UniTE',
 'UniTE-ref',
 'UniTE-src',
 'YISI-1',
 'chrF',
 'f101spBLEU',
 'f200spBLEU',
 'metricx_xl_DA_2019',
 'metricx_xl_MQM_2020',
 'metricx_xxl_DA_2019',
 'metricx_xxl_MQM_2020'}

In [72]:
WMT_scores = load_WMT_scores(WMT_scores_path, metrics_names_2022)


en-uk


100%|██████████| 24/24 [00:00<00:00, 151.52it/s]


en-ja


100%|██████████| 24/24 [00:00<00:00, 114.77it/s]


liv-en


...

100%|██████████| 24/24 [00:00<00:00, 139.66it/s]


ru-en


100%|██████████| 24/24 [00:00<00:00, 146.13it/s]


cs-en


100%|██████████| 24/24 [00:00<00:00, 97.36it/s] 


In [2]:
# Load the ACES scores
ACES_scores_2022_path = '/home/arnisa/Desktop/work/metric_sensitivity_analysis/scored_aces_final.tsv'
ACES_scores_2023_path = '/home/arnisa/Desktop/work/metric_sensitivity_analysis/aces-scored-2023-all-scores.quote_errors_removed.tsv'
# 2023 later
ACES_scores_2022 = load_ACES_scores(ACES_scores_2022_path, good_token='.-good', bad_token='.-bad', metric_mapping=METRIC_NAMES_MAPPING)
ACES_scores_2023 = load_ACES_scores(ACES_scores_2023_path, good_token='.-good', bad_token='.-bad', metric_mapping=METRIC_NAMES_MAPPING, skip_metrics=[])

# Load WMT scores - metrics for 2022
metrics_names_2022 = set(ACES_scores_2022.keys())
WMT_scores_path = '/home/arnisa/Desktop/work/metric_sensitivity_analysis/mt-metrics-eval-v2/wmt22/metric-scores'
WMT_scores = load_WMT_scores(WMT_scores_path, metrics_names_2022)
metrics_names_2022 = list(set(ACES_scores_2022.keys()).intersection(set(WMT_scores.keys())))
sensitivities_2022, _, _, phenomena_2022 = calculate_sensitivities(ACES_scores_2022, WMT_scores, mapping=PHENOMENA_MAPPING)

# Load WMT scores - metrics for 2023
metrics_names_2023 = set(ACES_scores_2023.keys())
WMT_scores_path = '/home/arnisa/Desktop/work/metric_sensitivity_analysis/mt-metrics-eval-v2/wmt22/metric-scores'
WMT_scores = load_WMT_scores(WMT_scores_path, metrics_names_2023)
metrics_names_2023 = list(set(ACES_scores_2023.keys()).intersection(set(WMT_scores.keys())))
sensitivities_2023, _, _, phenomena_2023 = calculate_sensitivities(ACES_scores_2023, WMT_scores, mapping=PHENOMENA_MAPPING)

# load the ACES scores from the paper
ACES_summary_2022 = load_ACES_scores_summary_2022()
ACES_summary_2023 = load_ACES_scores_summary_2023(skip_metrics=[])

INFO:logger:Loading ACES the scores...
INFO:logger:Loading ACES the scores...


en-uk


100%|██████████| 24/24 [00:00<00:00, 135.85it/s]


en-ja


100%|██████████| 24/24 [00:00<00:00, 81.19it/s]


...

100%|██████████| 43/43 [00:00<00:00, 268.23it/s]


ru-en


100%|██████████| 43/43 [00:00<00:00, 216.05it/s]


cs-en


100%|██████████| 43/43 [00:00<00:00, 182.55it/s]


In [3]:
# From the ACES 2022 Paper:
METRICS_GROUPING_2022= {"baseline": ["BLEU", "f101spBLEU", "f200spBLEU", "chrF", "BERTScore", "BLEURT-20",
                                     "COMET-20", "COMET-QE", "YISI-1"],
                        "reference-based": ["COMET-22", 'metricx_xl_DA_2019', 'metricx_xl_MQM_2020', 'metricx_xxl_DA_2019', 'metricx_xxl_MQM_2020',
                                            'MS-COMET-22', "UniTE", "UniTE-ref"],
                        "reference-free": ["COMETKiwi", "Cross-QE", 'HWTSC-Teacher-Sim', 'HWTSC-TLM',
                                           'KG-BERTScore', "MS-COMET-QE-22", "UniTE-src"]

                    }

METRICS_GROUPING_2023 = {
    "baseline": ["BERTScore", "BLEU", "BLEURT-20", "chrF", "COMET-22", "COMETKiwi", 
                 "f200spBLEU", "MS-COMET-QE-22", "Random-sysname", "YiSi-1"],
    "reference-based": ["eBLEU", "embed_llama", "MaTESe", "MetricX-23", "MetricX-23-b", 
                        "MetricX-23-c", "partokengram_F", "tokengram_F", "XCOMET-Ensemble",
                        "XCOMET-XL", "XCOMET-XXL", "XLsim"],
    "reference-free": ["cometoid22-wmt21", "cometoid22-wmt22", "cometoid22-wmt23", "CometKiwi-XL", 
                       "CometKiwi-XXL", "GEMBA-MQM", "KG-BERTScore", "MetricX-23-QE", "MetricX-23-QE-b", 
                       "MetricX-23-QE-c", "XCOMET-QE-Ensemble", "XLsimQE"]                           
}
# From the ACES 2022 Paper:
METRICS_GROUPING_SHORT_2022 = {"baseline": ["BLEU", "BERTScore", "BLEURT-20", "YISI-1"],
                        "reference-based": ["COMET-22", 'metricx_xl_MQM_2020', "UniTE", "UniTE-ref"],
                        "reference-free": ["COMETKiwi", 'KG-BERTScore', "UniTE-src"]
                    }

In [7]:
WMT_scores.keys()

dict_keys(['MS-COMET-QE-22', 'COMET', 'BERTScore', 'chrF', 'KG-BERTScore', 'YiSi-1', 'BLEURT-20', 'BLEU', 'COMETKiwi', 'f200spBLEU', 'MATESE', 'MEE4'])

In [35]:
import decimal
import re

def format_number(number:float, max_phenomena:bool = False, dec:str = '0.000', color='\colorbox[HTML]{B2EAB1}') -> str:
    if number == -np.inf:
        return '----'
    if number < 0:
        out = '-' + str(decimal.Decimal(-number).quantize(decimal.Decimal(dec)))
    else:
        out = '\phantom{-}' + str(decimal.Decimal(number).quantize(decimal.Decimal(dec)))
    if max_phenomena:
        return color + '{\\textbf{' + out + '}}'
    else:
        return out
    
def format_metric(metric:str):
    return re.sub(r'_', '\\_', metric)

def find_max_on_col(scores:Dict[str, Dict[str, Dict[str, int]]], metrics_names:List[str] = metrics_names_2022, phenomena:List[str]=PHENOMENA, k_highest:int=1) -> Dict[str,str]:
    max_metrics = {str(k): [[] for metric in metrics_names] for k in range(k_highest)}
    avgs = []
    for i,p in enumerate(phenomena):
        col = []
        for metric in metrics_names:
            if metric not in scores and metric in METRIC_NAMES_MAPPING:
                metric = METRIC_NAMES_MAPPING[metric]
            elif metric not in scores and metric in METRIC_MAPPING_BACK:
                metric = METRIC_MAPPING_BACK[metric]
            if metric not in scores:
                col.append(-np.inf)
            else:
                col.append(scores[metric][p])
        for k in range(k_highest):
            max_ids = np.where(col == np.partition(col, -k-1)[-k-1])[0]
            for max_id in max_ids:
                max_metrics[str(k)][max_id].append(i)
        col = np.array(col)
        avgs.append(np.average(col[col > -np.inf]))
    return max_metrics, avgs

def make_header(phenomena:List[str]=PHENOMENA) -> str:
    res = "\\begin{sidewaystable*}[ht] \n \small \n \setlength{\\tabcolsep}{3.75pt} \n \centering \n \\begin{tabular}{@{}lccccccccccc@{}} \n \\\\\\toprule \n"
    for p in PHENOMENA:
        if p in phenomena:
            res += " & "
            res += format_metric(PHENOMENA_HEADER_1[p])
    res += """ & \\textbf{ACES-} \\\\\n"""
    for p in PHENOMENA:
        if p in phenomena:
            res += " & "
            res += format_metric(PHENOMENA_HEADER_2[p])
    res += """ & \\textbf{Score}\\\\\n\midrule\n\\textit{\\textbf{Examples}} """
    for p in PHENOMENA:
        if p in phenomena:
            res += " & "
            res += '\\textit{' + str(NUM_SAMPLES[p]) + '}'
    res += """\\\\ \n \midrule"""
    return res

def generate_summary_table(scores:Dict[str, Dict[str, Dict[str, int]]], metrics_groups:Dict[str,list] = METRICS_GROUPING_2022, phenomena:List[str]=PHENOMENA, k_highest:int=1) -> str:
    out = ''
    metrics_names = []
    for group in metrics_groups.values():
        for metric in group:
            if metric not in scores and metric in METRIC_NAMES_MAPPING:
                metrics_names.append(METRIC_NAMES_MAPPING[metric])
            elif metric not in scores and metric in METRIC_MAPPING_BACK:
                metrics_names.append(METRIC_MAPPING_BACK[metric])
            else:
                metrics_names.append(metric)

    max_in_columns, avgs = find_max_on_col(scores, metrics_names=metrics_names, phenomena=phenomena, k_highest=k_highest)
    aces_scores_col = []
    for group, metrics in metrics_groups.items():
        for metric in metrics:
            if metric not in scores and metric in METRIC_NAMES_MAPPING:
                metric = METRIC_NAMES_MAPPING[metric]
            elif metric not in scores and metric in METRIC_MAPPING_BACK:
                metric = METRIC_MAPPING_BACK[metric]
            row = {}
            for p_id, p in enumerate(phenomena):
                if metric not in scores:
                    row[p] = 0.0
                else:
                    row[p] = scores[metric][p]
            aces_scores_col.append(comp_aces_score(row))
    max_aces_ids = np.where(list(aces_scores_col) == np.max(aces_scores_col))[0]

    m_id = 0
    for group, metrics in metrics_groups.items():
        for metric in metrics:
            if metric not in scores and metric in METRIC_NAMES_MAPPING:
                metric = METRIC_NAMES_MAPPING[metric]
            elif metric not in scores and metric in METRIC_MAPPING_BACK:
                metric = METRIC_MAPPING_BACK[metric]
            out += format_metric(metric) + '\t\t\t\t\t'
            for p_id, p in enumerate(phenomena):
                if metric not in scores:
                    out += '&\t ---- \t' 
                else:
                    if k_highest == 1:
                        max_ids = max_in_columns['0'][metrics_names.index(metric)]
                        out += '&\t' + format_number(scores[metric][p], max_phenomena=p_id in max_ids) + '\t'   
                    else:
                        for k in range(k_highest):
                            max_ids = max_in_columns[str(k)][metrics_names.index(metric)]
                            if p_id in max_ids:
                                color=COLORS[k]
                                break
                        out += '&\t' + format_number(scores[metric][p], max_phenomena=True, color=color) + '\t'   
                            
                                 
            out += '&\t' + format_number(aces_scores_col[m_id], dec='0.00', max_phenomena=m_id in max_aces_ids) + '\t \\\\ \n'
            m_id += 1
        out += '\midrule \n'
    out += 'Average\t\t\t\t\t'
    for p_id, p in enumerate(phenomena):
        out += '&\t' + format_number(avgs[p_id], max_phenomena=False) + '\t'
    out += '\\\\ \n\\bottomrule\t\t\t\t\t'
    return out

In [12]:
def comp_aces_score(overview_results: List[float]) -> float:
    '''
    Compute weighted summary score based on top-level correlation scores.
    '''
    for p in PHENOMENA:
        if p not in overview_results:
            overview_results[p] = [0]
    aces_score = 5 * np.mean(overview_results['addition']) + \
                 5 * np.mean(overview_results['omission']) + \
                 5 * np.mean(overview_results['mistranslation']) +\
                 1 * np.mean(overview_results['untranslated']) + \
                 1 * np.mean(overview_results['do not translate']) + \
                 5 * np.mean(overview_results['overtranslation']) + \
                 5 * np.mean(overview_results['undertranslation']) + \
                 1 * np.mean(overview_results['real-world knowledge']) + \
                 1 * np.mean(overview_results['wrong language']) + \
                 0.1 * np.mean(overview_results['punctuation'])
    return aces_score

In [67]:
sensitivities_2022.keys()

dict_keys(['chrF', 'Cross-QE', 'BLEURT-20', 'metricx_xxl_DA_2019', 'UniTE-src', 'metricx_xl_DA_2019', 'HWTSC-Teacher-Sim', 'UniTE-ref', 'COMET-20', 'COMET-22', 'BERTScore', 'COMET-QE', 'f101spBLEU', 'UniTE', 'metricx_xxl_MQM_2020', 'f200spBLEU', 'metricx_xl_MQM_2020', 'COMETKiwi', 'HWTSC-TLM', 'MS-COMET-QE-22', 'MS-COMET-22', 'KG-BERTScore', 'BLEU'])

In [34]:
np.partition([1,-2,3], -2)[-2]

1

In [36]:
METRIC_MAPPING_BACK = {'YISI-1':'YiSi-1', 'BERTScore':'BERTscore', 'COMETKiwi':'CometKiwi', 'MATESE':'MaTESe', 'COMET-22':'COMET'}
METRIC_NAMES_MAPPING = {
    # For ACES 2022
    'COMET-QE-Baseline':'COMET-QE',
    'YiSi-1':'YISI-1',
    # For ACES 2023
    'BERTscore':'BERTScore',
    'COMET':'COMET-22',
    'CometKiwi':'COMETKiwi',
    'MaTESe':'MATESE',  
    'spBLEU':'f200spBLEU'
}
PHENOMENA_HEADER_1 = dict(zip(PHENOMENA, ['\\hyperref[sec:addition-omission]{\\textbf{addition}}', '\hyperref[sec:addition-omission]{\\textbf{omission}}', '\hyperref[sec:source-disambig]{\\textbf{mistranslation}}', 
                      '\hyperref[sec:untranslated]{\\textbf{untranslated}}', '\hyperref[sec:do-not-translate]{\\textbf{do not}}', 
                      '\hyperref[sec:overtranslation_undertranslation]{\textbf{overtranslation}}', '\hyperref[sec:overtranslation_undertranslation]{\\textbf{undertranslation}}',
                        '\hyperref[sec:real-world-knowledge]{\\textbf{real-world}}', '\hyperref[sec:wrong_language]{\textbf{wrong}}', '\hyperref[sec:punctuation]{\\textbf{punctuation}}',
                          '\\textbf{ACES-}\\']))
PHENOMENA_HEADER_2 = dict(zip(PHENOMENA, ['', '',  '',  '', '\\hyperref[sec:do-not-translate]{\\textbf{translate}}', '',  '', '\hyperref[sec:real-world-knowledge]{\\textbf{knowledge}}',
                            '\hyperref[sec:wrong_language]{\\textbf{language}}', '', '\\textbf{Score}']))
NUM_SAMPLES = dict(zip(PHENOMENA, [999, 999, 24457, 1300, 100, 1000, 1000, 2948, 2000, 1673]))
COLORS = ['\colorbox{green1}', '\colorbox{green2}', '\colorbox{green3}', '\colorbox{green4}', '\colorbox{green5}', '\colorbox{white}', '\colorbox{red1}', '\colorbox{red2}', '\colorbox{red3}', '\colorbox{red4}', '\colorbox{red5}']
phenomena = ["addition", 'mistranslation', 'untranslated', 'do not translate', 'real-world knowledge']
print(make_header(phenomena=phenomena))
print(generate_summary_table(sensitivities_2022, metrics_groups=METRICS_GROUPING_SHORT_2022, phenomena=phenomena, k_highest=11))

\begin{sidewaystable*}[ht] 
 \small 
 \setlength{\tabcolsep}{3.75pt} 
 \centering 
 \begin{tabular}{@{}lccccccccccc@{}} 
 \\\toprule 
 & \hyperref[sec:addition-omission]{\textbf{addition}} & \hyperref[sec:source-disambig]{\textbf{mistranslation}} & \hyperref[sec:untranslated]{\textbf{untranslated}} & \hyperref[sec:do-not-translate]{\textbf{do not}} & \hyperref[sec:real-world-knowledge]{\textbf{real-world}} & \textbf{ACES-} \\
 &  &  &  & \hyperref[sec:do-not-translate]{\textbf{translate}} & \hyperref[sec:real-world-knowledge]{\textbf{knowledge}} & \textbf{Score}\\
\midrule
\textit{\textbf{Examples}}  & \textit{999} & \textit{24457} & \textit{1300} & \textit{100} & \textit{2948}\\ 
 \midrule
BLEU					&	\colorbox{green1}{\textbf{\phantom{-}0.215}}	&	\colorbox{red5}{\textbf{-0.088}}	&	\colorbox{green4}{\textbf{\phantom{-}1.194}}	&	\colorbox{red5}{\textbf{\phantom{-}0.253}}	&	\colorbox{red5}{\textbf{-0.306}}	&	\phantom{-}1.77	 \\ 
BERTScore					&	\colorbox{green5}{\textbf{\phantom{-}0.113}

In [9]:
'f200spBLEU' in ACES_scores_2023

False

# Plots

In [26]:
# create groups here:
means = {metric:[sensitivities_2022[metric][p] for p in PHENOMENA] for metric in metrics_names_2022}
tau = {metric:[ACES_summary_2022[metric][p] for p in PHENOMENA] for metric in metrics_names_2022 if metric in ACES_summary_2022}

In [21]:
# Group the metrics:
bleus = ["BLEU", "f101spBLEU", "f200spBLEU"]
comets = ["COMET-20", "COMET-22", "MS-COMET-22", "MS-COMET-QE-22"]
xl = ["metricx_xl_DA_2019", "metricx_xl_MQM_2020", "metricx_xxl_DA_2019", "metricx_xxl_MQM_2020"]
unite = ["UniTE", "UniTE-src", "UniTE-ref"]

In [28]:
grouped_line_plot([means, tau], bleus, ["Mean(good-bad)", "tau"], PHENOMENA)

In [38]:
def grouped_line_plot(groups: List[Dict[str,list]], metrics_names: List[str], group_labels: List[str], phenomena: List[str]):
    '''
    Inputs: 
        1. means and tau scores
        format = {
            metric1: [score for phenomenon 1, score for phenomenon 2, ..]
        }
        2. A list of the labels for: 
            the groups (mean (good-bad), tau, ...)
            metrics
            phenomena (the order is important because in means and tau scores the scores are ordered acc. to phenomena)
    '''
    assert len(groups) > 0 and len(groups) == len(group_labels) and len(metrics_names) > 0
    fig = go.Figure()
    colors = [['lightsteelblue',  'aqua', 'aquamarine', 'darkturquoise'],
        ['chocolate', 'coral', 'crimson', 'orange']]
    for i,group in enumerate(groups):
        for j,metric in enumerate(metrics_names):
            fig.add_trace(go.Scatter(x=phenomena, y=group[metric],mode='lines',name=group_labels[i]+" - "+metric,
                          line=dict(color=colors[i][j])))
    fig.update_layout(
        title=" ".join(group_labels)
        )
    fig.show()